In [96]:
import os
import re
import sys
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
import squidpy as sq
from joblib import Parallel, delayed
from skimage import exposure, io
from tqdm.notebook import tqdm, trange

sc.settings.verbosity = 3

from functools import reduce

import anndata as ad
import scanorama
import seaborn as sns
from matplotlib import cm, colors
from matplotlib.pyplot import rc_context
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [97]:
import multiprocessing

n_cpu = multiprocessing.cpu_count()

In [98]:
p_dir = (Path().cwd().parents[0]).absolute()
data_dir = p_dir / "data"

In [99]:
%load_ext autoreload
%autoreload 2

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

import utils as my_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
donors = [
    "LN Donor A",
    "LN Donor E",
    "INT Donor B",
    "INT Donor E",
    "TS Donor A",
    "TS Donor E",
    "SP Donor A"
]


# Import data

In [101]:
import napari
import matplotlib.patches as mpatches
from skimage.segmentation import mark_boundaries
import collections

def get_imgs(file_path, name):
    f = h5py.File(file_path, "r")
    imgs = f[name]
    labels = list(f[name].attrs["labels"])
    return imgs, labels


def contrast_streching(img):
    p2, p98 = np.percentile(img, (1, 99))
    return exposure.rescale_intensity(img, in_range=(p2, p98))


# Read mask image
def get_masks(mask_folder):
    """
    Function to get all mask from mask forlder
    """
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tif" in name:
                filename = os.path.join(dirpath, name)
                img = skimage.io.imread(filename)
                condition = int(name.split("_")[0])
                masks[condition] = img
    return collections.OrderedDict(sorted(masks.items()))

def get_imgs(imgs_folder):
    imgs = {}
    for (dirpath, dirnames, filenames) in os.walk(imgs_folder):
        for name in sorted(filenames):
            if "png" in name:
                img = io.imread(os.path.join(dirpath, name))
                condition = int(name.split(".")[0])
                imgs[condition] = img
    return collections.OrderedDict(sorted(imgs.items()))    

In [102]:
nuclei_image_path = p_dir / 'data' / 'ROI_nuclei' 
nuclei_mask_path =  p_dir / 'data' / 'masks_nuclei_v2'

In [103]:
donor = donors[5]

In [104]:
masks = get_masks(nuclei_mask_path / donor)
imgs = get_imgs(nuclei_image_path / donor)

In [105]:
# viewer = napari.view_image(np.stack(imgs.values()))
# viewer.add_labels(np.stack(masks.values()))

# Expand nuclei

In [106]:
import matplotlib.colors as mcolors
from scipy.ndimage import distance_transform_edt
from skimage.segmentation import mark_boundaries

def expand_labels(label_image, distance=1):
    distances, nearest_label_coords = distance_transform_edt(
        label_image == 0, return_indices=True
    )
    labels_out = np.zeros_like(label_image)
    dilate_mask = distances <= distance
    # build the coordinates to find nearest labels,
    # in contrast to [1] this implementation supports label arrays
    # of any dimension
    masked_nearest_label_coords = [
        dimension_indices[dilate_mask]
        for dimension_indices in nearest_label_coords
    ]
    nearest_labels = label_image[tuple(masked_nearest_label_coords)]
    labels_out[dilate_mask] = nearest_labels
    return labels_out

def save_mask(mask, folder, region):
    folder.mkdir(parents=True, exist_ok=True)
    path = os.path.join(folder, f'{region}' + ".tif")
    skimage.io.imsave(path, mask)        


In [107]:
cyto_mask_path = p_dir / 'data' / 'masks'

In [108]:
masks_expanded = collections.OrderedDict()    
for region in masks.keys():
    mask = expand_labels(masks[region], distance=2)
    masks_expanded[region] = mask

In [109]:
viewer = napari.view_image(np.stack(imgs.values()))
viewer.add_labels(np.stack(masks_expanded.values()))

C:\Users\thu71\AppData\Roaming\Python\Python39\site-packages\napari\plugins\_plugin_manager.py:549: UserWarning: Plugin 'napari-crop' has already registered a function widget 'crop region' which has now been overwritten
  warn(message=warn_message)


<Labels layer 'Labels' at 0x26a9e427490>

In [110]:
for region, mask in masks_expanded.items():
    save_mask(mask, cyto_mask_path / donor, region)